In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder  # label 수로 변환

In [2]:
# 데이터 폴더 경로
path = 'C:\\Users\\JungHyeona\\Documents\\Project\\05_AI\\PW'

In [3]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label

In [4]:
images = []
labels = []

for for_name in os.listdir(path):  # i = ['BackgroundNoise', 'LoopNoise', 'Normal(Air)']
    forder = os.path.join(path, for_name)  # ex) path/Normal(Air)

    for j in os.listdir(forder):
        file = os.path.join(forder, j)  # ex) path/Normal(Air)/a.png
        
        image = Image.open(file)
        label = os.path.basename(os.path.dirname(file))
        
        images.append(image)
        labels.append(label)

In [5]:
# 라벨 값 변경
for i in range(len(labels)):
    if labels[i] == 'BackgroundNoise' or labels[i] == 'LoopNoise':
        labels[i] = 0
    else:
        labels[i] = 1

In [6]:
len(labels)

1851

In [7]:
# 데이터 전처리
transform = transforms.Compose([
     transforms.Resize((32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((-0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [8]:
# 데이터셋 인스턴스 생성
dataset = CustomDataset(images, labels, transform=transform)

In [9]:
# 데이터셋을 train, validation, test로 분리
train_dataset, test_dataset = train_test_split(dataset, test_size=0.1, random_state=42)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, random_state=42)

In [10]:
# DataLoader를 통해 데이터를 불러올 수 있도록 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 모델

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.model = nn.Sequential(
            # (32, 32, 3) -> (32, 32, 32), (16, 16, 32)
            nn.Conv2d(3, 32, kernel_size=(3, 3), padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            
            # (16, 16, 32) -> (16, 16, 64) -> (8, 8, 64)
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            
            # (8, 8, 64) -> (8, 8, 128) -> (4, 4, 128)
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            
            nn.Flatten(),
            nn.Linear(4*4*128, 18),
            nn.Linear(18, 3)
        )
        

    def forward(self,x):
    	# self.layer에 정의한 연산 수행
        out = self.model(x)
        return out

In [12]:
model = CNN()

# GPU 사용 가능 여부에 따라 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model.to(device)

Using cuda device


CNN(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=2048, out_features=18, bias=True)
    (11): Linear(in_features=18, out_features=3, bias=True)
  )
)

In [13]:
loss_func = nn.CrossEntropyLoss()
learning_rate = 0.0002
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [15]:
total_batch = len(train_loader)
print('총 배치의 수 : {}'.format(total_batch))

for epoch in range(20):
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss_func(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

총 배치의 수 : 47
[Epoch:    1] cost = 0.626494884
[Epoch:    2] cost = 0.407184452
[Epoch:    3] cost = 0.324663699
[Epoch:    4] cost = 0.305074394
[Epoch:    5] cost = 0.274699867
[Epoch:    6] cost = 0.284021676
[Epoch:    7] cost = 0.244960889
[Epoch:    8] cost = 0.251282334
[Epoch:    9] cost = 0.272711843
[Epoch:   10] cost = 0.227181688
[Epoch:   11] cost = 0.22448191
[Epoch:   12] cost = 0.243052915
[Epoch:   13] cost = 0.215338856
[Epoch:   14] cost = 0.211994022
[Epoch:   15] cost = 0.22361429
[Epoch:   16] cost = 0.212352261
[Epoch:   17] cost = 0.211287037
[Epoch:   18] cost = 0.208230466
[Epoch:   19] cost = 0.190289646
[Epoch:   20] cost = 0.20020175


In [16]:
# 모델 평가 함수
def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [17]:
# 모델 평가
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
val_accuracy = evaluate(model, val_loader, device)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 97.60%


In [19]:
# 모델 저장
torch.save(model, path+'cnn.pt')

# test